In [7]:
#thread;;

#require "core";;
#require "core.syntax";;
#require "stdio";;

open Printf;;
open Stdio;;
open Core;;

In [17]:
let executable = "/home/igandhi/Documents/build/bin/klee"

let klee_stats = "/home/igandhi/Documents/build/bin/klee-stats"

let output_flags =
  [ 
  "search=random-path"; "no-forking"; "track-instruction-time"; 
  "libc=uclibc"; "posix-runtime"; "write-paths"; "output-conditions"
  ]
  
let replay_flags =
  [ 
  "search=random-path"; "no-forking"; "track-instruction-time"; 
  "libc=uclibc"; "posix-runtime"
  ]

let stats_flags = [ "print-all"; "to-csv" ]

let directory = "../klee/examples/llvm-coreutils/bitcodes/"

let bitcode = "ls.bc"

let bitcodes = ["ls.bc" ; "cat.bc" ; "env.bc" ; "split.bc" ; "tee.bc" ]

(* number of paths to explore with klee *)
let path_n = 1

val executable : string = "/home/igandhi/Documents/build/bin/klee"


val klee_stats : string = "/home/igandhi/Documents/build/bin/klee-stats"


val output_flags : string list =
  ["search=random-path"; "no-forking"; "track-instruction-time";
   "libc=uclibc"; "posix-runtime"; "write-paths"; "output-conditions"]


val replay_flags : string list =
  ["search=random-path"; "no-forking"; "track-instruction-time";
   "libc=uclibc"; "posix-runtime"]


val stats_flags : string list = ["print-all"; "to-csv"]


val directory : string = "../klee/examples/llvm-coreutils/bitcodes/"


val bitcode : string = "ls.bc"


val bitcodes : string list =
  ["ls.bc"; "cat.bc"; "env.bc"; "split.bc"; "tee.bc"]


val path_n : int = 1


In [15]:
exception FileError of string

type stderrOutput = 
  | Print
  | Ignore
  | Keep

let flags_to_str (flags : string list) : string =
  String.concat ~sep:" " @@ List.map flags (fun x -> "--" ^ x) 

(** run runs the given command, and returns each line of output of standard out*)
let run (cmd : string) (suppress : stderrOutput) : string list =
  let cmd = match suppress with
  | Print -> cmd
  | Ignore -> cmd ^ " 2> /dev/null"
  | Keep -> cmd ^ " 2>&1"
  in
  let inp = Unix.open_process_in @@ cmd in
  let r = In_channel.input_lines inp in
  In_channel.close inp;
  r
  
(* Outputs are logged as log1.csv, log2.csv... lowest_unused finds the lowest unused log number*)
let lowest_unused () : int = 
    let filenames = run ("ls logs/") Print in
    let num_to_name n = "log" ^ string_of_int n ^ ".csv" in
    let is_unused n = not @@ List.mem filenames (num_to_name n) (String.equal) in
    match List.find (List.init 100 Fun.id) is_unused with
    | Some unused -> unused
    | None -> raise (FileError "All file numbers used")

let write_to_log (headers : string list) (values : string list) (bitcode : string) : unit =
  let i = lowest_unused () in
  let oc = Out_channel.create ~append:true @@ "logs/log" ^ string_of_int i ^ ".csv" in
  fprintf oc "#%s\n %s\n" bitcode (Option.value (List.hd headers) ~default:"No headers");
  List.iter values (fprintf oc "%s\n");
  Out_channel.close oc

(** get_csv_stats returns the results of "klee-stats" as a tuple of (header, value) *)
let get_csv_stats (directory : string) (cond : string) : string * string =
  let command =
    klee_stats ^ " " ^ directory ^ "klee-last " ^ flags_to_str stats_flags
  in
  let csv_table = run command Print in
  ((Option.value (List.hd csv_table) ~default:"No header") ^ ",Cond", 
  (Option.value (List.hd @@ List.rev csv_table) ~default:"No rows") ^ "," ^ cond)
  
let print_str_list = fun l -> List.iter l print_endline

exception FileError of string


type stderrOutput = Print | Ignore | Keep


val flags_to_str : string list -> string = <fun>


val run : string -> stderrOutput -> string list = <fun>


val lowest_unused : unit -> int = <fun>


val write_to_log : string list -> string list -> string -> unit = <fun>


val get_csv_stats : string -> string -> string * string = <fun>


val print_str_list : string list -> unit = <fun>


In [10]:
let clear_subdirs (directory : string) : unit = ( run ("rm -rf " ^ directory ^ "*/") Print : string list) |> ignore

val clear_subdirs : string -> unit = <fun>


In [11]:
(*  Given KLEE output, returns a string of where the output is located *)
let get_output_dir (klee_output : string list) : string = 
  let output_str = String.concat ~sep:"\n" klee_output in
  let r = Str.regexp "KLEE: output directory is \"\\(.*\\)\"" in
  Str.search_forward r output_str 0 |> ignore;
  Str.matched_group 1 output_str
 
(*  Given KLEE output, returns a list of replayposition, condition pairs *)
let get_branches (klee_output : string list) : (int * string) list =
  let output_str = String.concat ~sep:" " klee_output in
  let r = Str.regexp "forking on condition: \\([^,]+\\), line number: [0-9]+, position \\([0-9]+\\)" in
  
  let rec match_regs start =
    try
    let next_start = Str.search_forward r output_str start in
    let c = Str.matched_group 1 output_str in
    let p = Str.matched_group 2 output_str in
    (int_of_string p, c) :: match_regs (next_start + 1)
    with _ -> []
  in
  match_regs 0

val get_output_dir : string list -> string = <fun>


val get_branches : string list -> (int * string) list = <fun>


In [18]:
let main (bitcode : string) =
    let klee_command =
        String.concat ~sep:" " @@ [executable; flags_to_str output_flags; directory ^ bitcode; " --sym-arg 10"]
    in
    let initial_run = (run klee_command Keep : string list) in
    print_str_list initial_run;
    let output_dir = get_output_dir initial_run in
    let branches = get_branches initial_run in
    let run_branch (p, c) : string * string =
        let klee_command =
        String.concat ~sep:" "
        @@ [ executable; flags_to_str replay_flags; "--replace-condition=" ^ string_of_int p; 
        "--replay-path=\"" ^ output_dir ^ "/test000001.path\""; directory ^ bitcode; " --sym-arg 10" ]
        in
        print_endline klee_command;
        (run klee_command Print : string list) |> ignore;
        get_csv_stats directory c 
    in
    (* header strings, value strings *)
    let (h_str , v_str) = Caml.List.split @@ List.map ((0, "baseline")::branches) run_branch in
    print_endline @@ Caml.List.hd h_str;
    print_str_list v_str;
    clear_subdirs directory;
    write_to_log h_str v_str bitcode

let () = List.iter bitcodes main

val main : string -> unit = <fun>


KLEE: NOTE: Using POSIX model: /home/igandhi/Documents/build/Debug+Asserts/lib/libkleeRuntimePOSIX.bca
KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
KLEE: output directory is "/home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-0"
KLEE: Using STP solver backend
KLEE: WARNING: undefined reference to function: __ctype_get_mb_cur_max
KLEE: WARNING: undefined reference to function: bindtextdomain
KLEE: WARNING: undefined reference to function: cap_free
KLEE: WARNING: undefined reference to function: cap_get_file
KLEE: WARNING: undefined reference to function: cap_to_text
KLEE: WARNING: undefined reference to function: dcgettext
KLEE: WARNING: undefined reference to function: gettext
KLEE: WARNING: undefined reference to function: textdomain
KLEE: WARNING: executable has module level assembly (ignoring)
KLEE: WARNING ONCE: calling external: syscall(16, 0, 21505, 94630264991824) at klee/runtime/POSIX/fd.c:1007 1

KLEE: NOTE: Using POSIX model: /home/igandhi/Documents/build/Debug+Asserts/lib/libkleeRuntimePOSIX.bca
KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
KLEE: output directory is "/home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-1"
KLEE: Using STP solver backend
KLEE: WARNING: undefined reference to function: __ctype_get_mb_cur_max
KLEE: WARNING: undefined reference to function: bindtextdomain
KLEE: WARNING: undefined reference to function: cap_free
KLEE: WARNING: undefined reference to function: cap_get_file
KLEE: WARNING: undefined reference to function: cap_to_text
KLEE: WARNING: undefined reference to function: dcgettext
KLEE: WARNING: undefined reference to function: gettext
KLEE: WARNING: undefined reference to function: textdomain
KLEE: WARNING: executable has module level assembly (ignoring)
KLEE: WARNING ONCE: calling external: syscall(16, 0, 21505, 94371642659136) at klee/runtime/POSIX/fd.c:1007 1

/home/igandhi/Documents/build/bin/klee --search=random-path --no-forking --track-instruction-time --libc=uclibc --posix-runtime --replace-condition=845 --replay-path="/home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-0/test000001.path" ../klee/examples/llvm-coreutils/bitcodes/ls.bc  --sym-arg 10


KLEE: NOTE: Using POSIX model: /home/igandhi/Documents/build/Debug+Asserts/lib/libkleeRuntimePOSIX.bca
KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
KLEE: output directory is "/home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-2"
KLEE: Using STP solver backend
KLEE: WARNING: undefined reference to function: __ctype_get_mb_cur_max
KLEE: WARNING: undefined reference to function: bindtextdomain
KLEE: WARNING: undefined reference to function: cap_free
KLEE: WARNING: undefined reference to function: cap_get_file
KLEE: WARNING: undefined reference to function: cap_to_text
KLEE: WARNING: undefined reference to function: dcgettext
KLEE: WARNING: undefined reference to function: gettext
KLEE: WARNING: undefined reference to function: textdomain
KLEE: WARNING: executable has module level assembly (ignoring)
KLEE: WARNING ONCE: calling external: syscall(16, 0, 21505, 94069375307856) at klee/runtime/POSIX/fd.c:1007 1

...

KLEE: NOTE: Using POSIX model: /home/igandhi/Documents/build/Debug+Asserts/lib/libkleeRuntimePOSIX.bca
KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
KLEE: output directory is "/home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-29"
KLEE: Using STP solver backend
KLEE: WARNING: undefined reference to function: __ctype_get_mb_cur_max
KLEE: WARNING: undefined reference to function: bindtextdomain
KLEE: WARNING: undefined reference to function: gettext
KLEE: WARNING: undefined reference to function: posix_fadvise
KLEE: WARNING: undefined reference to function: textdomain
KLEE: WARNING: executable has module level assembly (ignoring)
KLEE: WARNING ONCE: calling external: syscall(16, 0, 21505, 94466985344368) at klee/runtime/POSIX/fd.c:1007 10
KLEE: WARNING ONCE: Alignment of memory from call "malloc" is not modelled. Using alignment of 8.
KLEE: WARNING ONCE: calling __klee_posix_wrapped_main with extra argument

/home/igandhi/Documents/build/bin/klee --search=random-path --no-forking --track-instruction-time --libc=uclibc --posix-runtime --replace-condition=12767 --replay-path="/home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-0/test000001.path" ../klee/examples/llvm-coreutils/bitcodes/tee.bc  --sym-arg 10


KLEE: NOTE: Using POSIX model: /home/igandhi/Documents/build/Debug+Asserts/lib/libkleeRuntimePOSIX.bca
KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
KLEE: output directory is "/home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-30"
KLEE: Using STP solver backend
KLEE: WARNING: undefined reference to function: __ctype_get_mb_cur_max
KLEE: WARNING: undefined reference to function: bindtextdomain
KLEE: WARNING: undefined reference to function: gettext
KLEE: WARNING: undefined reference to function: posix_fadvise
KLEE: WARNING: undefined reference to function: textdomain
KLEE: WARNING: executable has module level assembly (ignoring)
KLEE: WARNING ONCE: calling external: syscall(16, 0, 21505, 93877260660240) at klee/runtime/POSIX/fd.c:1007 10
KLEE: WARNING ONCE: Alignment of memory from call "malloc" is not modelled. Using alignment of 8.
KLEE: WARNING ONCE: calling __klee_posix_wrapped_main with extra argument

Instructions,FullBranches,PartialBranches,NumBranches,UserTime,NumStates,MallocUsage,NumQueries,NumQueryConstructs,NumObjects,WallTime,CoveredInstructions,UncoveredInstructions,QueryTime,SolverTime,CexCacheTime,ForkTime,ResolveTime,QueryCexCacheMisses,QueryCexCacheHits,Breaks,Returns,Switchs,IndirectBrs,Invokes,Resumes,Unreachables,CleanupRets,CatchRets,CatchPads,CatchSwitchs,FNegs,Adds,FAdds,Subs,FSubs,Muls,FMuls,UDivs,SDivs,FDivs,URems,SRems,FRems,Ands,Ors,Xors,Allocas,Loads,Stores,AtomicCmpXchgs,AtomicRMWs,Fences,GetElementPtrs,Truncs,ZExts,SExts,FPTruncs,FPExts,FPToUIs,FPToSIs,UIToFPs,SIToFPs,IntToPtrs,PtrToInts,BitCasts,AddrSpaceCasts,ICmps,FCmps,PHIs,Selects,Calls,Shls,LShrs,AShrs,VAArgs,ExtractElements,InsertElements,ShuffleVectors,ExtractValues,InsertValues,LandingPads,CleanupPads,Cond
172249,64,197,1507,937194.0,0,47468832,26,294,0,673103.0,4330,20822,291981,294648,292804,0,61,26,-1,23663,1080,3,0,0,0,0,0,0,0,0,0,8235,0,1247,0,81,0,5,0,0,1,0,0,125,16,17,5302,51978,34835,0,0,0,

172249,64,197,1507,923558.0,0,47473440,26,294,0,706900.0,4330,20822,280362,283012,281168,0,67,26,-1,23663,1080,3,0,0,0,0,0,0,0,0,0,8235,0,1247,0,81,0,5,0,0,1,0,0,125,16,17,5302,51978,34835,0,0,0,16540,601,5615,2508,0,0,0,0,0,0,2,98,145,0,12605,0,1261,3,6252,25,0,6,0,0,0,0,0,0,0,0,(Eq false      (Eq 255          (Read w8 (Extract w32 0 (Add w64 1                                           (SExt w64 (ZExt w32 (Eq 45 (Read w8 1 arg00))))))                   [10=0] @ arg00)))
172249,64,197,1507,706768.0,0,47471344,25,277,0,580298.0,4330,20822,291694,294326,292536,0,67,25,-1,23663,1080,3,0,0,0,0,0,0,0,0,0,8235,0,1247,0,81,0,5,0,0,1,0,0,125,16,17,5302,51978,34835,0,0,0,16540,601,5615,2508,0,0,0,0,0,0,2,98,145,0,12605,0,1261,3,6252,25,0,6,0,0,0,0,0,0,0,0,(Eq 0      (Read w8 (Extract w32 0 (Add w64 2                                       (SExt w64 (ZExt w32 (Eq 45 (Read w8 1 arg00))))))               [10=0] @ arg00))
172249,64,197,1507,855613.0,0,47473440,26,294,0,630024.0,4330,20822,267708,270